In [3]:
# This is the 3rd step!
# 1. The transit search was run from the command line with e.g.:
#
#    python k2_superpig_search.py --fl step0-inputs/alltest > step1-eebls-results/alltest_results_2016May19.txt
#
#    (If you need to create the input (alltest), there is a cell below to help
#     ince ls * > file won't work on more than a few 1000 files)
#
# 2. This notebook (Step2_K2_Identify_Candidates) identifies candidate planets
#
# 3. This notebook (Step3_K2_Candidate_Star_Info) uses several sources to compile
#     information on the stars, which you will need for the fitting step.
#
# Note: heavily edited on 4/13/16 to account for MAST *finally* having gotten Teff values
# 
%matplotlib inline

#import modules, including the codes used in this study
import numpy as np
import pylab
import matplotlib.pyplot as plt
from scipy import interpolate
import math
import string
import glob
import os
from uncertainties import ufloat
from uncertainties import umath

# our package
import k2_superpig_search as k2_sppg

In [4]:
######### Locations
# Analysis dir:
superpig_dir = os.getcwd()
# Results from previous steps:
step0_dir = "step0-inputs/"
step1_dir = "step1-eebls-results/"
step2_dir = "step2-candidate-results/"
# Results from this step should go:
results_dir = "step3-candidate-stars/"

In [5]:
### Read in a presorted folder and follow everything in it
def files_in_folder(folder,matchString="hlsp_k2sff_k2_lightcurve_*.png"):
    filelist = glob.glob(folder + matchString)
    objList=[]
    for ff in filelist:
        obj = string.replace(string.split(string.replace(ff,folder,""),"-")[0], "hlsp_k2sff_k2_lightcurve_","")
        objList.append(int(obj))
    return objList

In [6]:
###### We are ASSUMING that all transits have been identified in Step 2
###### and are located in the promisng directory 
follow_list=[]
all_quarters = ["c00", "c01", "c02", "c03", "c04", "c05"]
for qq in all_quarters: # can do more than one quarter at a time
    match = step2_dir+"promising-"+qq+"*"
    matching = glob.glob(match)
    promising_dir = matching[0]
    candidate_list = files_in_folder(promising_dir+"/_transit-like/")
    follow_list=follow_list+candidate_list
follow_list= sorted(follow_list)

print follow_list
print len(follow_list)

[201264302, 201606542, 201637175, 201650711, 202094740, 203533312, 205152172, 206103150, 206151047, 206169375, 206298289, 206417197, 210414957, 210605073, 210707130, 210754505, 210954046, 210961508, 211152484, 211357309, 211685045, 211995325, 212150006]
23


# Query MAST for star params (where available)

In [7]:
import kplr

In [8]:
#### Load Kepler API (the kplr package)
####   The API is a way to query an online database (e.g., MAST) for information in an automated way
####   Since values on MAST are updated regularly, it is recommended that you take caution when re-running this code
####   in case something unexpectedly changes because, say, the transit parameters are different than last time.
#### You can use the API to:
####    get information on planets and their host stars
####    download lightcurve data from the MAST database
#### Possible values for planet info are found at:
####     http://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html
client = kplr.API()

#star = client.k2_star(202137899)
#help(client.k2_star)

In [12]:
# I really reeally need to learn how to suppress these stupid 
# "Unrecognized parameter" warnings from MAST
# Because this doesn't work:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# One off checking of specific target
# Planet Hunters: 
#  http://talk.planethunters.org/#/boards/BPH0000007/discussions/DPH00019on?page=1&comment_id=563919c41cd2282bb3000137
# Planets: 211623903, 211685045, 212150006, 
# EBs: 211578677, 211685048, 211953866, 211978865, 211995966, 211999656, 212155299, 212158225 
# ("212158225 Period about 0.1333 days, depth 0.0005, duration about 1.5 hours. It coiuld also be a very short period PC.")
# Other: 211934173 looks like an EB, but the period is not constant, I don't know what's going on here???
# Not listed: 211357309, 211995325

#check=[211357309, 211578677, 211623903, 211685045, 211685048, 211934173, 211953866, 211978865, 211995325,
#       211995966, 211999656, 212150006, 212155299, 212158225]
#for cc in check:
#    star = client.k2_star(cc)
#    print cc, star.teff, star.kp


For every object in the followlist, get the information on MAST

In [11]:
epic_teff_list = []; epic_teff_err_list = [];
star_ra_list=[]; star_dec_list=[]; star_rad_list = []
kepmag_list=[]; bmag_list = []; vmag_list = []; jmag_list=[]; kmag_list=[]
objtype=[]
for ff in follow_list:
    star = client.k2_star(ff)
    star_ra_list.append(star.k2_ra)
    star_dec_list.append(star.k2_dec)

    star_rad_list.append(star.rad)
    epic_teff_list.append(star.teff)
    epic_teff_err_list.append(star.e_teff)
    kepmag_list.append(star.kp)
    #print ff, star.jmag - star.kmag
    bmag_list.append(star.bmag)
    vmag_list.append(star.vmag)
    jmag_list.append(star.jmag)
    kmag_list.append(star.kmag)
    objtype.append(star.objtype)
print follow_list, star_ra_list, star_dec_list
print epic_teff_list

[201264302, 201606542, 201637175, 201650711, 202094740, 203533312, 205152172, 206103150, 206151047, 206169375, 206298289, 206417197, 210414957, 210605073, 210707130, 210754505, 210954046, 210961508, 211152484, 211357309, 211685045, 211995325, 212150006] [169.598013, 170.311881, 169.482818, 172.044052, 100.46312, 243.955378, 245.180008, 331.203044, 333.71501, 344.016383, 337.357688, 337.133466, 64.325266, 65.62329, 59.464394, 64.64493, 60.903766, 59.920104, 60.069895, 133.23263, 123.359621, 131.207598, 128.169544] [-2.991577, 2.144426, 2.61907, 2.826891, 27.0972, -25.818471, -19.14414, -12.018893, -10.769168, -10.332234, -8.266702, -6.347505, 13.804842, 17.037875, 18.465254, 19.179056, 22.249157, 22.365984, 25.48, 10.944721, 15.713759, 20.177694, 23.031999]
[3299.0, 5307.0, 3865.0, 3718.0, None, 7072.0, 4202.0, 5950.0, 5928.0, 6167.0, 3724.0, 5007.0, 5404.0, None, 4268.0, 6041.0, None, 5056.0, 5983.0, 3563.0, 3832.0, 4071.0, 5528.0]


In [13]:
# Value names came from this random difference log:
# https://github.com/dfm/kplr/commit/d7a35e6329c2c0111c9c5d4600c2260c722f9eb5
print star.k2_ra, star.k2_dec, star.kp
print star.bmag, star.rmag, star.vmag, star.jmag, star.hmag, star.kmag
print star.teff, star.d, star.rad, star.e_rad, star.lum,  star.plx, star.logg
print star.e_teff
print star.objtype

128.169544 23.031999 14.7
15.678 14.616 14.771 13.462 13.054 12.948
5528.0 769.5 0.896 None None None 4.482
None
STAR


In [22]:
# What type of objects are our candidates?
#print objtype
for ii,oo in enumerate(objtype):
    if oo != "STAR":
        print follow_list[ii], oo

202094740 None


In [23]:
# Teff values have been propogated at last! (4/13/16)
# ... but not the errors, UGH.
for ii in range(len(follow_list)):
    print follow_list[ii], epic_teff_list[ii], epic_teff_err_list[ii], star_rad_list[ii]

201264302 3299.0 None 0.188
201606542 5307.0 None 0.851
201637175 3865.0 None 0.382
201650711 3718.0 None 0.34
202094740 None None None
203533312 7072.0 None 2.028
205152172 4202.0 None 0.531
206103150 5950.0 None 1.104
206151047 5928.0 None 1.226
206169375 6167.0 None 1.384
206298289 3724.0 None 0.333
206417197 5007.0 None 0.75
210414957 5404.0 None 2.319
210605073 None None None
210707130 4268.0 None 0.507
210754505 6041.0 None 1.318
210954046 None None None
210961508 5056.0 None 2.589
211152484 5983.0 None 1.214
211357309 3563.0 None 0.272
211685045 3832.0 None 0.376
211995325 4071.0 None 0.417
212150006 5528.0 None 0.896


## Hack the stupid Teff errors by use the ExoFOP query

https://exofop.ipac.caltech.edu/k2/search.php

We also get the log(g) and [Fe/H] values from ExoFOP currently, which come with errors. The catch is you have to run a query on a list of files every time your target list changes. Boo.

In [24]:
# https://exofop.ipac.caltech.edu/k2/search.php
# via Dan Huber (http://arxiv.org/abs/1512.02643)
exofop_stellar_param_file = "data_from_catalogs/exofop_search20160323_121749.csv"
gg = open(exofop_stellar_param_file,"r")
lines = gg.readlines()
gg.close()
exofop_values={}
for line in lines:
    if line[0]!="#":
        elems=line.rstrip().split(",")
        if elems[0]=="Target":
            headers=elems
        elif elems == ['']:
            foo=1
        else:
            obj = eval(elems[0])
            for ii,hh in enumerate(headers):
                exofop_values[hh,obj] = elems[ii]

In [25]:
print headers

['Target', 'RA', 'Dec', 'Teff (K)', 'Teff Uncertainty', 'log(g)', 'log(g) Uncertainty', 'Stellar Radius (R_Sun)', 'Stellar Radius Uncertainty', 'Vsin(i) (km/s)', 'Vsin(i) Uncertainty', 'Spectral Type', "logR'HK", "logR'HK Uncertainty", '[Fe/H]', '[Fe/H] Uncertainty', 'Distance (pc)', 'Distance Uncertainty', 'Stellar Mass (M_Sun)', 'Stellar Mass Uncertainty', 'Density (g/cm3)', 'Density Uncertainty', 'Rot Period (days)', 'Rot Period Uncertainty', 'Luminosity (L_Sun)', 'Luminosity Uncertainty', 'S-index', 'S-index Uncertainty']


In [26]:
exo_teff=[]; exo_teff_err=[]; exo_rstar=[]; exo_rstar_err=[]
exo_logg=[]; exo_logg_err=[]; exo_feh=[]; exo_feh_err=[]
for ff in follow_list:
    print ff,
    print "Teff", exofop_values["Teff (K)",ff],  exofop_values["Teff Uncertainty",ff],
    print "Rstar",exofop_values["Stellar Radius (R_Sun)",ff], exofop_values["Stellar Radius Uncertainty",ff],
    print "log g",exofop_values["log(g)",ff], exofop_values["log(g) Uncertainty",ff],
    print "[Fe/H]",exofop_values["[Fe/H]",ff], exofop_values["[Fe/H] Uncertainty",ff]

    exo_teff.append( exofop_values["Teff (K)",ff])
    exo_teff_err.append(exofop_values["Teff Uncertainty",ff] )
    exo_rstar.append( exofop_values["Stellar Radius (R_Sun)",ff])
    exo_rstar_err.append( exofop_values["Stellar Radius Uncertainty",ff] )
    exo_logg.append( exofop_values["log(g)",ff])
    exo_logg_err.append(exofop_values["log(g) Uncertainty",ff] )
    exo_feh.append( exofop_values["[Fe/H]",ff])
    exo_feh_err.append(exofop_values["[Fe/H] Uncertainty",ff] )
 

201264302 Teff 3299 32 Rstar 0.188 0.004 log g 5.106 0.026 [Fe/H] 0.155 0.05
201606542 Teff 5307 159 Rstar 0.851 0.061 log g 4.531 0.03 [Fe/H] 0.051 0.1
201637175 Teff 3865 90 Rstar 0.382 0.069 log g 4.886 0.084 [Fe/H] -0.048 0.15
201650711 Teff 3718 107 Rstar 0.34 0.047 log g 4.943 0.045 [Fe/H] -0.027 0.12
202094740 Teff   Rstar   log g   [Fe/H]  
203533312 Teff 7072 451 Rstar 2.028 0.925 log g 4.029 0.02 [Fe/H] 0.17 0.12
205152172 Teff 4202 242 Rstar 0.531 0.087 log g 4.758 0.042 [Fe/H] -0.04 0.2
206103150 Teff 5950 108 Rstar 1.104 0.319 log g 4.322 0.12 [Fe/H] -0.176 0.2
206151047 Teff 5928 120 Rstar 1.226 0.546 log g 4.247 0.18 [Fe/H] -0.169 0.24
206169375 Teff 6167 122 Rstar 1.384 0.415 log g 4.16 0.165 [Fe/H] -0.168 0.2
206298289 Teff 3724 59 Rstar 0.333 0.027 log g 4.942 0.055 [Fe/H] 0.017 0.05
206417197 Teff 5007 151 Rstar 0.75 0.039 log g 4.587 0.035 [Fe/H] -0.063 0.12
210414957 Teff 5404 86 Rstar 2.319 0.233 log g 3.779 0.02 [Fe/H] -0.395 0.24
210605073 Teff   Rstar   log g  

In [27]:
print exo_teff
print epic_teff_list

['3299', '5307', '3865', '3718', '', '7072', '4202', '5950', '5928', '6167', '3724', '5007', '5404', '', '4268', '6041', '', '5056', '5983', '3563', '3832', '4071', '5528']
[3299.0, 5307.0, 3865.0, 3718.0, None, 7072.0, 4202.0, 5950.0, 5928.0, 6167.0, 3724.0, 5007.0, 5404.0, None, 4268.0, 6041.0, None, 5056.0, 5983.0, 3563.0, 3832.0, 4071.0, 5528.0]


# Output a list of candidates for observation (optional)

In [28]:
import time as systime
now = systime.strftime("%Y%b%d")

In [29]:
from astropy import units as u
from astropy.coordinates import SkyCoord

c = SkyCoord(ra=10.68458*u.degree, dec=41.26917*u.degree)
print c.ra.hms
print c.dec.dms
print c.to_string('hmsdms')

hms_tuple(h=0.0, m=42.0, s=44.299200000000525)
dms_tuple(d=41.0, m=16.0, s=9.0120000000092659)
00h42m44.2992s +41d16m09.012s


In [30]:
# EPIC, RA, Dec, V, Kep, Dwarf status
export_for_obs = False
if export_for_obs:
    gg=open(results_dir+"obslist_"+now+".tsv","w")
    print >>gg, "\t".join(["Candidate","RA_hms","Dec_hms","V_mag","Kep_mag","Dwarf"])
    for ii,cc in enumerate(follow_list):
        c = SkyCoord(ra=star_ra_list[ii]*u.degree, dec=star_dec_list[ii]*u.degree)
        print >>gg, "\t".join([str(cc), c.to_string('hmsdms'), str(vmag_list[ii]),
                               str(kepmag_list[ii]),is_dwarf[ii]])

    gg.close()

# Read in Vanderbilt's catalog Teff (where available)

In [33]:
# This file is only C0-5!
tess_catalog_file = "data_from_catalogs/k2tess_teff_err.tsv"

gg = open(tess_catalog_file,"r")
lines = gg.readlines()
gg.close()
header = lines[0].rstrip().split("\t")
print header
teff_pos = header.index("teff")
teff_err_pos = header.index("tefferr")
epic_pos = header.index("k2name")
dwarf_pos=header.index("isdwarf")
#print len(lines), teff_pos,teff_err_pos, epic_pos
tess_epic_nums=[]; tess_teffs=[]; tess_teff_errs=[]; is_dwarf=[]
for line in lines[1:]:
    elems=line.rstrip().split("\t")
    tess_epic_nums.append( eval(elems[epic_pos]) )
    tess_teffs.append( elems[teff_pos] )
    tess_teff_errs.append( elems[teff_err_pos] )
    is_dwarf.append( elems[dwarf_pos] )

['k2name', 'k2camp', 'isdwarf', 'teff', 'tefferr', 'kepmag']


In [34]:
# About 2/3 of the 90K total C0-5 targets in here
print len(tess_epic_nums)

63191


# Get own Teffs from McDonald (Mike Endl)

In [35]:
gg=open("data_from_observations/McD_followup.dat","r")
lines = gg.readlines()
gg.close()
header_elems= lines[0].rstrip().split()
print header_elems
observed=[]; observed_teff=[]; observed_teff_err=[]; observed_logg=[]; observed_logg_err=[]
observed_feh=[]; observed_feh_err=[]
for line in lines[2:]:
    if line[0] != "#":
        elems=line.rstrip().split()
        obj = eval(elems[0])
        # If we are averaging two data points
        # This will dumbly break for three...
        if obj in observed:
            previous_ind = observed.index(obj)
            prev = observed_teff[previous_ind]
            prev_err = observed_teff_err[previous_ind]
            prev_u = ufloat(prev,prev_err) 
            this = eval(elems[header_elems.index("Teff[K]")])
            this_err = eval(elems[header_elems.index("Teff[K]")+1])
            this_u = ufloat(this,this_err) 
            ave_u = (this_u + prev_u) / 2.
            print "Averaged", obj, ave_u, "[From:",prev,this,"]"
            # Now replace with the average value
            observed_teff[previous_ind] = ave_u.nominal_value
            observed_teff_err[previous_ind] = round(ave_u.std_dev,1)
        # Most objects only have one data point
        else:
            observed.append(eval(elems[0]))
            observed_teff.append( eval(elems[header_elems.index("Teff[K]")]) ) 
            observed_teff_err.append( eval(elems[header_elems.index("Teff[K]")+1]) )

            observed_logg.append( eval(elems[header_elems.index("logg")]) )
            observed_logg_err.append( eval(elems[header_elems.index("logg")+1]) )

            observed_feh.append( eval(elems[header_elems.index("[Fe/H]")]) )
            observed_feh_err.append( eval(elems[header_elems.index("[Fe/H]")+1]) )
         
print observed
print observed_teff
print observed_teff_err
print observed_logg
print observed_feh

['EPIC#', 'HJD', 'S/N', 'RV[km/s]', 'err', 'Teff[K]', 'err', '[Fe/H]', 'err', 'logg', 'err', 'vsini[km/s]', 'err', 'T_exp[s]', 'Notes']
Averaged 210954046 (6.12+/-0.20)e+03 [From: 6062.0 6188.0 ]
[210414957, 210707130, 210754505, 210954046, 210961508, 211152484, 201606542, 203533312, 201650711]
[5838.0, 4462.0, 5875.0, 6125.0, 4925.0, 6188.0, 5540.0, 6620.0, 4340.0]
[102.0, 84.0, 103.0, 202.4, 45.0, 67.0, 108.0, 86.0, 103.0]
[4.12, 4.17, 4.04, 3.0, 3.42, 4.12, 4.81, 4.19, 4.25]
[0.3, -0.28, 0.04, 0.0, -0.06, -0.12, 0.03, 0.09, -0.81]


# Use Teff/R* values from the literature if available

In [36]:
# For 201637175 (Sanchis-Ojeda 15b) http://adsabs.harvard.edu/abs/2015ApJ...812..112S
# Teff 3830 ± 100
# logg 4.65 ± 0.12
# FeH 0.03 ± 0.08
# Mstar 0.60 ± 0.07
# Rstar 0.57 ± 0.06

# Sanchis-Ojeda 15 (p2) for WASP-47, aka 206103150,
# the results were Teff = 5565 ± 60K, logg = 4.29 ± 0.07, and [Fe/H] = 0.43 ± 0.04. 
# "These values are all in agreement with those presented by
#   Hellier et al. (2012) and Mortier et al. (2013)."
lit_teff_objects = [201637175, 206103150]
lit_teffs = [3830, 5565.0]
lit_teff_errs = [100, 60.0]
lit_logg = [4.65, 4.29]
lit_logg_err = [0.12, 0.07]
lit_feh = [0.03, 0.43]
lit_feh_err = [0.08, 0.04]

In [38]:
# Objects that have a published value for R* (not just for T_eff)
has_lit_rstar_objects = [206103150]
has_lit_rstar_ref = "Mortier2013" # also Becker2016
has_lit_rstar = [1.15] # the last column, from the LC, NOT the previous column
has_lit_rstar_err = [0.04]

# Special cases which have no Teff values

In [39]:
# Option A: based on V-K, J-H, H-K colors assuming it's a main-sequence (dwarf)
# Allen's Astrophysical Quantities, p 151, Table 7.6, G0 to M6
# 211357309 checks out: between M0/M1, 3800-3680
# 211623903: K2/K4

allen_vMinusK=[1.41, 1.46, 1.53, 1.64, 1.96, 2.22, 2.63, 2.85, 3.16, 3.65, 3.87, 4.11, 4.65, 5.28, 6.17, 7.37]
allen_jMinusH=[0.31, 0.32, 0.33, 0.37, 0.45, 0.50, 0.58, 0.61, 0.66, 0.67, 0.66, 0.66, 0.64, 0.62, 0.62, 0.66]
allen_hMinusK=[0.05, 0.05, 0.06, 0.06, 0.08, 0.09, 0.11, 0.11, 0.15, 0.17, 0.18, 0.20, 0.23, 0.27, 0.29, 0.36]
allen_teff   =[5930, 5830, 5740, 5620, 5240, 5010, 4560, 4340, 4040, 3800, 3680, 3530, 3380, 3180, 3030, 2850]
allen_jMinusK=np.array(allen_jMinusH) + np.array(allen_hMinusK)
print allen_jMinusK

interp_vk =interpolate.interp1d( allen_vMinusK, allen_teff)
interp_jk =interpolate.interp1d( allen_jMinusK, allen_teff)
assign_teff = float(interp_vk(4.023))
assign_teff2 = float(interp_jk(0.896))
print assign_teff, assign_teff2

[ 0.36  0.37  0.39  0.43  0.53  0.59  0.69  0.72  0.81  0.84  0.84  0.86
  0.87  0.89  0.91  1.02]
3584.375 3135.0


In [42]:
# Option B: 
# Sloan photometry from Pinsonneault 2012ApJS..199...30P
# 210605073
# g-r = -0.037, g-i=-0.145, and g-z=-0.266
# Just off edge of table; we used the radius of a F0 star from Allen's.
infer_teff_objects = [210605073]
infer_teffs = [7020.0]
infer_teff_err = [500.0] # made up!

# 2016-May-20: email from JJ Hermes re 210605073
# the Vanderburg & Johnson aperture is mostly composed of the nearby star 
#   SDSS J042229.72+170208.1 (r=17.22 mag)
#  https://www.cfa.harvard.edu/~avanderb/k2c4/ep210605073.html
#  which has colors from
#   http://skyserver.sdss.org/dr12/en/tools/explore/Summary.aspx?id=1237671256905678933
u=21.66
g=18.80
r=17.22
i=16.48
z=16.03
print g-r, g-i, g-z, i-z
# 1.58 2.32 2.77 0.45
# That makes it an M dwarf, off the *bottom* edge of the table (M<0.58 and T<4047.1)

1.58 2.32 2.77 0.45


In [85]:
# Dilution factor
mag1=17.22 # Star with transits, SDSS J042229.72+170208.1 (r=17.22 mag)
mag2=17.90 # WD, EPIC 210605073 (SDSS J042229.58+170216.3, r=17.90 mag)
flux1 = 10**((mag1)/-2.5)
flux2 = 10**((mag2)/-2.5)
# What fraction of the total light is due to star 1, the non-white-dwarf?
frac1 = flux1/(flux1+flux2)
frac2 = flux2/(flux1+flux2)
print frac1, frac2
# What does this mean for the transit depth (radius ratio^2)?
adj_radius = math.sqrt(1/frac1)
print adj_radius

0.651650739753 0.348349260247
1.23877534662


In [296]:
# Option C: Infer Teff from B-V
# Boyajian 2012b has Teff from B-V (for G-K-M stars)

def teff_from_bminusv(bMinusV):
    # from Boyajian 2012a, but NOT good for Mdwarfs!
    # log_teff =3.9680- 0.2633*bMinusV - 3.2195*bMinusV**2 + 15.3548*bMinusV**3 - 27.2901*bMinusV**4 + 19.9193*bMinusV**5 - 4.5127*bMinusV**6
    # teff = 10**log_teff
    if bMinusV == None:
        return None
    else:
        teff = 8010 - 4095*bMinusV + 819*bMinusV**2 +133*bMinusV**3 + 39*bMinusV**4 - 362*bMinusV**5
        return round(teff)

# Which objects have which Teff values?

In [297]:
catalog_used=[]; follow_teff=[]; follow_teff_err=[]
follow_logg=[]; follow_logg_err=[]; follow_feh=[]; follow_feh_err=[]

#print "Object", "EPIC", "TESS","McD","Lit"
print "Object","Best","Err","Source"
for ii,ff in enumerate(follow_list):
    # Okay, here's the order of preference
    #(1) McDonal Teff (measured by us)
    #(2) Lit Teff (measured)
    #(3) EPIC catalog
    #(4) TESS/Vanderbilt
    #(5) Infer from photometry if all else fails
    if ff in observed:
        obs_ind = observed.index(ff)
        best = observed_teff[obs_ind]
        best_err = observed_teff_err[obs_ind]
        best_logg = observed_logg[obs_ind]
        best_logg_err = observed_logg_err[obs_ind]
        best_feh = observed_feh[obs_ind]
        best_feh_err = observed_feh_err[obs_ind]
        source = "McDonald"
    elif ff in lit_teff_objects:
        lit_ind = lit_teff_objects.index(ff)
        best = lit_teffs[lit_ind]
        best_err = lit_teff_errs[lit_ind]
        best_logg = lit_logg[lit_ind]
        best_logg_err = lit_logg_err[lit_ind]
        best_feh = lit_feh[lit_ind]
        best_feh_err = lit_feh_err[lit_ind]
        source = "Literature spectra"
    elif epic_teff_list[ii] != None:
        best = epic_teff_list[ii]
     #   best_err = epic_teff_err_list[ii]  # Maybe when MAST gets these values again
        best_err = eval(exo_teff_err[ii]) # From ExoFOP instead (note that it's a fixed file)
        best_logg = eval(exo_logg[ii])
        best_logg_err = eval(exo_logg_err[ii])
        best_feh = eval(exo_feh[ii])
        best_feh_err = eval(exo_feh_err[ii])
        source = "EPIC"
    elif ff in tess_epic_nums:
        tess_ind = tess_epic_nums.index(ff)
        best = eval(tess_teffs[tess_ind])
        best_err = eval(tess_teff_errs[tess_ind])
        best_logg = None
        best_logg_err = None
        best_feh = None
        best_feh_err = None
        source = "TESS/Vanderbilt"
    elif ff in infer_teff_objects:
        infer_ind = infer_teff_objects.index(ff)
        best = infer_teffs[infer_ind]
        best_err = infer_teff_err[infer_ind]
        best_logg = None
        best_logg_err = None
        best_feh = None
        best_feh_err = None
        source = "Inferred from photometry"
    else:
        best = None
        best_err = None
        best_logg = None
        best_logg_err = None
        best_feh = None
        best_feh_err = None
        source = "none"
        
    catalog_used.append(source)
    follow_teff.append(best)
    follow_teff_err.append(best_err)
    follow_logg.append(best_logg)
    follow_logg_err.append(best_logg_err)
    follow_feh.append(best_feh)
    follow_feh_err.append(best_feh_err)
    print ff, best, best_err, source, best_logg, best_logg_err, best_feh, best_feh_err

Object Best Err Source
201264302 3299.0 32 EPIC 5.106 0.026 0.155 0.05
201606542 5540.0 108.0 McDonald 4.81 0.12 0.03 0.04
201637175 3830 100 Literature spectra 4.65 0.12 0.03 0.08
201650711 4340.0 103.0 McDonald 4.25 0.42 -0.81 0.08
202094740 6481.0 25.0 TESS/Vanderbilt None None None None
203533312 6620.0 86.0 McDonald 4.19 0.12 0.09 0.05
205152172 4202.0 242 EPIC 4.758 0.042 -0.04 0.2
206103150 5565.0 60.0 Literature spectra 4.29 0.07 0.43 0.04
206151047 5928.0 120 EPIC 4.247 0.18 -0.169 0.24
206169375 6167.0 122 EPIC 4.16 0.165 -0.168 0.2
206298289 3724.0 59 EPIC 4.942 0.055 0.017 0.05
206417197 5007.0 151 EPIC 4.587 0.035 -0.063 0.12
210414957 5838.0 102.0 McDonald 4.12 0.14 0.3 0.07
210605073 7020.0 500.0 Inferred from photometry None None None None
210707130 4462.0 84.0 McDonald 4.17 0.18 -0.28 0.1
210754505 5875.0 103.0 McDonald 4.04 0.19 0.04 0.05
210954046 6125.0 202.4 McDonald 3.0 0.58 0.0 0.16
210961508 4925.0 45.0 McDonald 3.42 0.08 -0.06 0.07
211152484 6188.0 67.0 McDonal

# Beyond Teff: R*

Which stars already have nominal radius values from EPIC catalog?

In [298]:
for ii in range(len(follow_list)):
    print follow_list[ii], exo_rstar[ii],exo_rstar_err[ii]

201264302 0.188 0.004
201606542 0.851 0.061
201637175 0.382 0.069
201650711 0.34 0.047
202094740  
203533312 2.028 0.925
205152172 0.531 0.087
206103150 1.104 0.319
206151047 1.226 0.546
206169375 1.384 0.415
206298289 0.333 0.027
206417197 0.75 0.039
210414957 2.319 0.233
210605073  
210707130 0.507 0.076
210754505 1.318 0.313
210954046  
210961508 2.589 0.512
211152484 1.214 0.347
211357309 0.272 0.028
211685045 0.376 0.083
211995325 0.417 0.156
212150006 0.896 1.298


# Derive R* using Boyajian 2012b

In [299]:
# Can get R* directly from Teff (Boyajian 2012b eqn 9 and Table 10)
# Note that eqn 8 in that paper is only from 3200-5500 K, but the one we use
# has had the sun added in to extend to g like stars
#  if 3200 <= teff <= 5500:
#      -10.8828 + 7.18727*10**-3*teff - 1.50957*10**-6*teff**2+ 1.07572*10**-10*teff**3
def rstar_from_teff(teff):    
    return -8.133 + 5.09342*10**-3*teff - 9.86602*10**-7*teff**2 + 6.47963*10**-11*teff**3

In [300]:
def rstar_from_teff_err(teff, teff_err):
    nom=rstar_from_teff(teff)
    err1=rstar_from_teff(teff+teff_err)
    err2=rstar_from_teff(teff-teff_err)
    # We are adding in 0.031 R_star for the median absolute error reported in Boyajian2012b
    model_mad = 0.031
    max_err = max(abs(nom-err1), abs(nom-err2))
    return max_err+model_mad

In [301]:
# Hmm, the sun still comes out a bit small
print rstar_from_teff(5778), rstar_from_teff_err(5778,200)

0.858010142089 0.0733766371575


In [332]:
follow_rstar=[]; follow_rstar_err=[]
boyajian_rstar_for_epic_teff=[]; boyajian_rstar_err=[]
for ii in range(len(follow_list)):
    # A separate list to compare estimate R* for the EPIC values where available
    if exo_teff[ii] != "":
        boyajian_rstar_for_epic_teff.append(rstar_from_teff(eval(exo_teff[ii])))
        boyajian_rstar_err.append(rstar_from_teff_err(eval(exo_teff[ii]),eval(exo_teff_err[ii])))
    else:
        boyajian_rstar_for_epic_teff.append(0.)
        boyajian_rstar_err.append(0.)
    # First off, defer to literature value if we've selected to above:
    if follow_list[ii] in has_lit_rstar_objects:
        ind = has_lit_rstar_objects.index(follow_list[ii])
        follow_rstar.append(has_lit_rstar[ind])
        follow_rstar_err.append(has_lit_rstar_err[ind])
    # Otherwise using our best Teff
    else:
        follow_rstar.append(rstar_from_teff(follow_teff[ii]))
        follow_rstar_err.append(rstar_from_teff_err(follow_teff[ii],follow_teff_err[ii]))

Compare R* from EPIC to that derived from the EPIC Teff  (NOT the best Teff)

In [333]:
sig_dif_obj=[]
for ii in range(len(follow_list)):
    print follow_list[ii], "EPIC",exo_rstar[ii], exo_rstar_err[ii],
    print "Boyajian",round(boyajian_rstar_for_epic_teff[ii],3), round(boyajian_rstar_err[ii],3),
    if exo_rstar[ii] != "":
        boyajian_u = ufloat(boyajian_rstar_for_epic_teff[ii], boyajian_rstar_err[ii])
        exo_u = ufloat(exo_rstar[ii], exo_rstar_err[ii])
        print "Diff",boyajian_u - exo_u 
        if abs((boyajian_u - exo_u).nominal_value) > 3. * (boyajian_u - exo_u).std_dev:
            sig_dif_obj.append(follow_list[ii])
    else:
        print ""

201264302 EPIC 0.188 0.004 Boyajian 0.259 0.054 Diff 0.07+/-0.05
201606542 EPIC 0.851 0.061 Boyajian 0.796 0.048 Diff -0.06+/-0.08
201637175 EPIC 0.382 0.069 Boyajian 0.556 0.066 Diff 0.17+/-0.10
201650711 EPIC 0.34 0.047 Boyajian 0.496 0.082 Diff 0.16+/-0.09
202094740 EPIC   Boyajian 0.0 0.0 
203533312 EPIC 2.028 0.925 Boyajian 1.463 0.504 Diff -0.6+/-1.1
205152172 EPIC 0.531 0.087 Boyajian 0.657 0.099 Diff 0.13+/-0.13
206103150 EPIC 1.104 0.319 Boyajian 0.894 0.058 Diff -0.21+/-0.32
206151047 EPIC 1.226 0.546 Boyajian 0.889 0.061 Diff -0.3+/-0.5
206169375 EPIC 1.384 0.415 Boyajian 0.953 0.073 Diff -0.4+/-0.4
206298289 EPIC 0.333 0.027 Boyajian 0.499 0.058 Diff 0.17+/-0.06
206417197 EPIC 0.75 0.039 Boyajian 0.769 0.045 Diff 0.02+/-0.06
210414957 EPIC 2.319 0.233 Boyajian 0.806 0.041 Diff -1.51+/-0.24
210605073 EPIC   Boyajian 0.0 0.0 
210707130 EPIC 0.507 0.076 Boyajian 0.672 0.067 Diff 0.16+/-0.10
210754505 EPIC 1.318 0.313 Boyajian 0.916 0.066 Diff -0.40+/-0.32
210954046 EPIC   Boya

In [334]:
# Only two are significant, and, surprise, both are subgiants (according to EPIC):
for ii in range(len(follow_list)):
    if follow_list[ii] in sig_dif_obj:
        print follow_list[ii], "EPIC",exo_rstar[ii], exo_logg[ii],
# Did McDonald agree?
        if follow_list[ii] in observed:
            ind = observed.index(follow_list[ii])
            print "Measured",observed_logg[ind],observed_logg_err[ind]
        else:
            print ""

210414957 EPIC 2.319 3.779 Measured 4.12 0.14
210961508 EPIC 2.589 3.64 Measured 3.42 0.08


## What is the maximum planetary-sized radius ratio around this star?

In [335]:
max_planet_radius = 0.2 # 2 RJup
for ii in range(len(follow_list)):
    if follow_rstar[ii] > 0:
        print follow_list[ii], follow_teff[ii], follow_rstar[ii],  "Max rr:",max_planet_radius/follow_rstar[ii]

201264302 3299.0 0.259075765996 Max rr: 0.771974944208
201606542 5540.0 0.821562607583 Max rr: 0.24343853792
201637175 3830 0.542810926818 Max rr: 0.368452420758
201650711 4340.0 0.686073165935 Max rr: 0.291514097811
202094740 6481.0 1.07594893782 Max rr: 0.185882427102
203533312 6620.0 1.14674209075 Max rr: 0.174407132706
205152172 4202.0 0.656802281036 Max rr: 0.304505641613
206103150 5565.0 1.15 Max rr: 0.173913043478
206151047 5928.0 0.888598201688 Max rr: 0.225073604268
206169375 6167.0 0.953279914184 Max rr: 0.209801965849
206298289 3724.0 0.4989364803 Max rr: 0.400852629336
206417197 5007.0 0.769196661508 Max rr: 0.260011528922
210414957 5838.0 0.869434370778 Max rr: 0.230034614138
210605073 7020.0 1.41884402849 Max rr: 0.140959820801
210707130 4462.0 0.707382933232 Max rr: 0.282732294779
210754505 5875.0 0.877005997852 Max rr: 0.228048611401
210954046 6125.0 0.940308208398 Max rr: 0.21269621834
210961508 4925.0 0.761943528123 Max rr: 0.262486644506
211152484 6188.0 0.9600446999

In [336]:
for ii in range(len(follow_list)):
    print follow_list[ii],"R_*",follow_rstar[ii], follow_rstar_err[ii]

201264302 R_* 0.259075765996 0.0537375590613
201606542 R_* 0.821562607583 0.0459564596966
201637175 R_* 0.542810926818 0.072237722951
201650711 R_* 0.686073165935 0.052275464597
202094740 R_* 1.07594893782 0.0429235831576
203533312 R_* 1.14674209075 0.0805435146063
205152172 R_* 0.656802281036 0.0985627582391
206103150 R_* 1.15 0.04
206151047 R_* 0.888598201688 0.0607772926098
206169375 R_* 0.953279914184 0.0730290168372
206298289 R_* 0.4989364803 0.0579483977368
206417197 R_* 0.769196661508 0.0446522201427
210414957 R_* 0.869434370778 0.0529156822613
210605073 R_* 1.41884402849 0.54414689078
210707130 R_* 0.707382933232 0.0452490818372
210754505 R_* 0.877005997852 0.054380781395
210954046 R_* 0.940308208398 0.100649216187
210961508 R_* 0.761943528123 0.0351337641298
211152484 R_* 0.960044699962 0.053875628072
211357309 R_* 0.420851206262 0.0622145342864
211685045 R_* 0.543584998997 0.125426308748
211995325 R_* 0.623049262806 0.217028343337
212150006 R_* 0.820039786177 0.0541295340304


# Limb darkening (Claret 2011 tables, interpolated)

In [324]:
### Read in ld params for Kep filter and chosen microturbulent velocity
# Takes 5-10 sec because full file is BIG (lots of filters/velocities)
# We are assuming Fe/H == Z = 0 and microturbulent velocity = 2 km/s (by default)
xi = 2.0

# Other values are possible in the full file
quad_claret_file = superpig_dir+"data_from_catalogs/claret2011_table-af.tsv"
gg = open(quad_claret_file,"r")
lines = gg.readlines()
gg.close()
claret_logg_grid=[]; claret_teff_grid=[]; claret_feh_grid=[]
claret_u1_quad_grid=[]; claret_u2_quad_grid=[]
micros=[]
for line in lines[53:]: ## lots of headers
    elems = line.rstrip().split("\t")
    filt = elems[6]
    metallicity = eval(elems[2]) # Oh dear, these were originally reversed...
    microturb = eval(elems[3])
    if filt == "Kp":
        micros.append(microturb)
        if microturb == xi:
            claret_logg_grid.append(eval(elems[0]))
            claret_teff_grid.append(eval(elems[1]))
            claret_feh_grid.append(eval(elems[2]))
            claret_u1_quad_grid.append(eval(elems[4]))
            claret_u2_quad_grid.append(eval(elems[5]))

In [325]:
print len(claret_logg_grid), len(lines)
print "Microturb", set(micros)
all_logg = sorted(list(set(claret_logg_grid)))
all_feh = sorted(list(set(claret_feh_grid)))
all_teff = sorted(list(set(claret_teff_grid)))
print "logg",all_logg
print "Fe/H",all_feh
#print "Teff",all_teff
print min(claret_teff_grid), max(claret_teff_grid)

15670 446329
Microturb set([0.0, 1.0, 2.0, 4.0, 8.0])
logg [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
Fe/H [-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.5, 1.0]
2000 50000


In [326]:
# Following http://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html#scipy.interpolate.griddata
# because griddata does not require regular spacing
# and the different Teffs are defined for differnt logg/FeH values
from scipy.interpolate import griddata

# We need arrays for each point at which we have data in Teff, logg, and Fe/H space
grid_points = []
for ii in range(len(claret_teff_grid)):
    grid_points.append(np.array([claret_teff_grid[ii],
                                 claret_logg_grid[ii], claret_feh_grid[ii]]))

In [327]:
# Takes ~1sec per obj
follow_u1=[]; follow_u2=[]
for ii in range(len(follow_list)):
    obj = follow_list[ii]
    feh = follow_feh[ii]
    teff = follow_teff[ii]
    logg = follow_logg[ii]
    # Will have problems if numbers are slightly out of bounds
    if teff < min(all_teff):
        teff = min(all_teff) # Mdwarfs!
    if logg > max(all_logg):
        logg = max(all_logg) # there are some 5.1s
    # Will also have a problem with nans
    if logg == None:
        logg = 4.0
    if feh == None:
        feh = 0.0
    
    grid_u1 = griddata(grid_points, np.array(claret_u1_quad_grid),
                       np.array([teff, logg, feh]), method="linear")
    grid_u2 = griddata(grid_points, np.array(claret_u2_quad_grid),
                       np.array([teff, logg, feh]), method="linear")
    follow_u1.append(grid_u1[0])
    follow_u2.append(grid_u2[0])
    print obj, teff, logg, feh, grid_u1, grid_u2

201264302 3299.0 5.0 0.155 [ 0.416774] [ 0.3719065]
201606542 5540.0 4.81 0.03 [ 0.484344] [ 0.192602]
201637175 3830 4.65 0.03 [ 0.59762] [ 0.157]
201650711 4340.0 4.25 -0.81 [ 0.512576] [ 0.219108]
202094740 6481.0 4.0 0.0 [ 0.297] [ 0.3125768]
203533312 6620.0 4.19 0.09 [ 0.296744] [ 0.30762]
205152172 4202.0 4.758 -0.04 [ 0.7146192] [ 0.0665752]
206103150 5565.0 4.29 0.43 [ 0.5246762] [ 0.1718304]
206151047 5928.0 4.247 -0.169 [ 0.3878728] [ 0.2572662]
206169375 6167.0 4.16 -0.168 [ 0.3258132] [ 0.298718]
206298289 3724.0 4.942 0.017 [ 0.35498167] [ 0.35813367]
206417197 5007.0 4.587 -0.063 [ 0.5711226] [ 0.1407397]
210414957 5838.0 4.12 0.3 [ 0.4643408] [ 0.2162188]
210605073 7020.0 4.0 0.0 [ 0.29476] [ 0.28246]
210707130 4462.0 4.17 -0.28 [ 0.668604] [ 0.089691]
210754505 5875.0 4.04 0.04 [ 0.432598] [ 0.2301915]
210954046 6125.0 3.0 0.0 [ 0.35825] [ 0.2623]
210961508 4925.0 3.42 -0.06 [ 0.5599495] [ 0.1535065]
211152484 6188.0 4.12 -0.12 [ 0.340992] [ 0.278184]
211357309 3563.0 

In [328]:
print follow_u1

[0.41677399999999998, 0.484344, 0.59761999999999982, 0.51257600000000003, 0.29699999999999999, 0.29674400000000001, 0.71461920000000001, 0.52467620000000004, 0.38787279999999996, 0.32581319999999997, 0.35498166666666658, 0.57112260000000004, 0.46434080000000005, 0.29476000000000002, 0.66860399999999998, 0.43259800000000004, 0.35825000000000001, 0.55994949999999999, 0.34099199999999996, 0.39863999999999999, 0.42941499999999982, 0.45245879999999994, 0.43845179999999995]


# Output stellar properties for later reference

In [337]:
star_param_file = results_dir+"star_params_all.tsv"

In [338]:
gg = open(star_param_file,"w")
print >>gg, "\t".join(["Candidate","RA_degrees","Dec_degrees",
                       "Kepmag","Teff","Teff_err",
                       "logg","logg_err","Fe/H","Fe/H_err",
                       "Rstar","Rstar_err","ld_u1","ld_u2","Is_dwarf","Source"])
for ii in range(len(follow_list)):
    if follow_feh[ii] == None:
        follow_feh[ii] = 0.0
        follow_feh_err[ii] = 0.0
    if follow_logg[ii] == None:
        follow_logg[ii] = 4.0
        follow_logg_err[ii] = 0.0
    
    print >>gg, "\t".join([str(follow_list[ii]), str(star_ra_list[ii]), str(star_dec_list[ii]),
                           str(kepmag_list[ii]), str(follow_teff[ii]), str(follow_teff_err[ii]),
                           str(follow_logg[ii]), str(follow_logg_err[ii]), 
                           str(follow_feh[ii]), str(follow_feh_err[ii]), 
                           str(round(follow_rstar[ii],2)),str(round(follow_rstar_err[ii],2)),
                           str(round(follow_u1[ii],4)), str(round(follow_u2[ii],4)),
                           str(is_dwarf[ii]), catalog_used[ii]])
gg.close()

In [339]:
follow_list

[201264302,
 201606542,
 201637175,
 201650711,
 202094740,
 203533312,
 205152172,
 206103150,
 206151047,
 206169375,
 206298289,
 206417197,
 210414957,
 210605073,
 210707130,
 210754505,
 210954046,
 210961508,
 211152484,
 211357309,
 211685045,
 211995325,
 212150006]

# Some quick numbers

In [340]:
# Data from https://exofop.ipac.caltech.edu/k2/edit_target.php?id=203533312
vmag = 12.536# ± 0.020
jmag = 10.367# ± 0.024
kmag = 9.847# ± 0.024
vk = vmag-kmag
jk = jmag-kmag
vk_teff = float(interp_vk(vk))
jk_teff = float(interp_jk(jk))
print vk_teff, jk_teff


4501.0 5278.0


In [341]:
# https://exofop.ipac.caltech.edu/k2/edit_target.php?id=210605073
umag = 18.215# ± 0.014	 	2015-04-09 12:34:05	EPIC
gmag = 17.859# ± 0.006	 	2015-04-09 12:34:05	EPIC
rmag = 17.896# ± 0.006	 	2015-04-09 12:34:05	EPIC
Kepmag = 17.887#	 	2015-04-09 12:34:05	EPIC
imag = 18.004# ± 0.008	 	2015-04-09 12:34:05	EPIC
zmag = 18.125# ± 0.020	 	2015-04-09 12:34:05	EPIC
gr = ufloat(gmag,0.006) - ufloat(rmag,0.006)
gi = ufloat(gmag,0.006) - ufloat(imag,0.008)
gz = ufloat(gmag,0.006) - ufloat(zmag,0.020)
print gr, gi, gz

-0.037+/-0.008 -0.145+/-0.010 -0.266+/-0.021


In [342]:
# Compare to fiducial Teff table from ugriz photometry (http://arxiv.org/pdf/1110.4456.pdf)
# Those are all off the table for Table 1 (Fe/H=-0.2), i.e., M > 1.5 M_sun
table1_gr =  0.032
table1_gi = -0.022
table1_gz = -0.111

print gr - table1_gr, gi - table1_gi, gz - table1_gz

-0.069+/-0.008 -0.123+/-0.010 -0.155+/-0.021


In [343]:
1.3*11

14.3

In [344]:
# How much did 201650711 increase by?
orig_rp = 0.69 
(0.69/0.57)*orig_rp

0.8352631578947368

In [345]:
0.37491*24/2

4.49892